In [1]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('/data/arpit.goel/32_RecruitRestarauntPrediction/')

df_air_reserve=pd.read_csv('01.RawData/air_reserve.csv',parse_dates=['visit_datetime','reserve_datetime'])
df_air_store_info=pd.read_csv('01.RawData/air_store_info.csv')
df_air_visit_data=pd.read_csv('01.RawData/air_visit_data.csv',parse_dates=['visit_date'])
df_date_info=pd.read_csv('01.RawData/date_info.csv',parse_dates=['calendar_date'])
df_hpg_reserve=pd.read_csv('01.RawData/hpg_reserve.csv',parse_dates=['visit_datetime','reserve_datetime'])
df_hpg_store_info=pd.read_csv('01.RawData/hpg_store_info.csv')
df_store_id_relation=pd.read_csv('01.RawData/store_id_relation.csv')
df_sample_submission=pd.read_csv('01.RawData/sample_submission.csv')

In [2]:
df_air_visit_data.loc[:,'log_visitors']=np.log(1+df_air_visit_data.loc[:,'visitors'])
daily_visits=df_air_visit_data.groupby(['air_store_id','visit_date'])['log_visitors'].max().unstack()

In [49]:
def get_features_window(x,window):
    indices=x.notnull()*np.arange(len(x))
    indices=indices[indices>0]
    start=np.min(indices)
    print (x[start:])
    print (pd.rolling_apply(x[start:],7,lambda x: np.median(x[pd.notnull(x)])))

    
daily_visits.head(1).apply(get_features_window,axis=1,window=28)

visit_date
2016-07-01    3.583519
2016-07-02    2.302585
2016-07-03         NaN
2016-07-04    3.044522
2016-07-05    3.258097
2016-07-06    3.401197
2016-07-07    3.555348
2016-07-08    3.761200
2016-07-09    2.484907
2016-07-10         NaN
2016-07-11    3.258097
2016-07-12    3.218876
2016-07-13    3.610918
2016-07-14    3.367296
2016-07-15    3.367296
2016-07-16    1.791759
2016-07-17         NaN
2016-07-18         NaN
2016-07-19    3.258097
2016-07-20    2.995732
2016-07-21    3.871201
2016-07-22    3.988984
2016-07-23    2.833213
2016-07-24         NaN
2016-07-25    2.833213
2016-07-26    2.833213
2016-07-27    3.218876
2016-07-28    3.555348
2016-07-29    3.465736
2016-07-30    3.044522
                ...   
2017-03-24    3.555348
2017-03-25    2.079442
2017-03-26         NaN
2017-03-27    3.784190
2017-03-28    3.433987
2017-03-29    3.970292
2017-03-30    3.526361
2017-03-31    3.663562
2017-04-01    2.079442
2017-04-02         NaN
2017-04-03    3.526361
2017-04-04    2.890372


/data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_apply is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).apply(func=<function>,args=<tuple>,kwargs=<dict>)


air_store_id
air_00a91d42b08b08d9    None
dtype: object

In [57]:
rolling_features={}
from itertools import product

for date in pd.date_range('2016-01-01','2017-04-22'):


def get_rolling_values(arr,window):
    medians=np.nanmedian(arr,axis=1)
    means=np.nanmean(arr,axis=1)
    return pd.DataFrame([medians,means]).T
    
get_rolling_values(daily_visits.iloc[:5,365:385].values,5)

,0,1
0,3.135494,3.246440
1,2.249905,2.281453
2,2.079442,2.203517
3,2.047172,2.103251
4,1.791759,1.798716


In [109]:
from itertools import product
from datetime import timedelta,datetime
rolling_features={}
#for date,window in product(pd.date_range('2016-01-01','2016-04-22'),[7,14,28,56,84,91,182,364]):
for date,window in product(pd.date_range('2016-01-01','2016-01-03'),[7,14,28,56,84,91,182,364]):
    start=max(date-timedelta(days=window),datetime(2016,1,1))
    end=max(date-timedelta(days=1),datetime(2016,1,1))
    rolling_features[(date,'mean_global',window)]=daily_visits.loc[:,start:end].apply(lambda x: np.nanmean(x),axis=1)
    rolling_features[(date,'median_global',window)]=daily_visits.loc[:,start:end].apply(lambda x: np.nanmedian(x),axis=1)
    weekdays=pd.date_range(start,end)[::-window].sort_values()
    rolling_features[(date,'mean_local',window)]=daily_visits.loc[:,weekdays].apply(lambda x: np.nanmean(x),axis=1)
    rolling_features[(date,'median_local',window)]=daily_visits.loc[:,weekdays].apply(lambda x: np.nanmedian(x),axis=1)
    

/data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: RuntimeWarning: Mean of empty slice
/home/arpit.goel2/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:4016: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
/data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: Mean of empty slice


In [112]:
pd.DataFrame(rolling_features).unstack().unstack(level=(1,2))

mean_global                                \
                                        7         14        28        56    
           air_store_id                                                     
2016-01-01 air_00a91d42b08b08d9         NaN       NaN       NaN       NaN   
           air_0164b9927d20bcc3         NaN       NaN       NaN       NaN   
           air_0241aa3964b7f861         NaN       NaN       NaN       NaN   
           air_0328696196e46f18         NaN       NaN       NaN       NaN   
           air_034a3d5b40d5b1b1         NaN       NaN       NaN       NaN   
           air_036d4f1ee7285390         NaN       NaN       NaN       NaN   
           air_0382c794b73b51ad         NaN       NaN       NaN       NaN   
           air_03963426c9312048         NaN       NaN       NaN       NaN   
           air_04341b588bde96cd    2.397895  2.397895  2.397895  2.397895   
           air_049f6d5b402a31b2         NaN       NaN       NaN       NaN   
           air_04cae7c1bc9b2a0b         NaN       NaN       NaN       NaN   
           air_0585011fa179bcce         NaN       NaN       NaN       NaN   
           air_05c325d315cc17f5    3.401197  3.401197  3.401197  3.401197   
           air_0647f17b4dc041c8         NaN       NaN       NaN       NaN   
           air_064e203265ee5753         NaN       NaN       NaN       NaN   
           air_066f0221b8a4d533         NaN       NaN       NaN       NaN   
           air_06f95ac5c33aca10         NaN       NaN       NaN       NaN   
           air_0728814bd98f7367         NaN       NaN       NaN       NaN   
           air_0768ab3910f7967f         NaN       NaN       NaN       NaN   
           air_07b314d83059c4d2         NaN       NaN       NaN       NaN   
           air_07bb665f9cdfbdfb         NaN       NaN       NaN       NaN   
           air_082908692355165e         NaN       NaN       NaN       NaN   
           air_083ddc520ea47e1e         NaN       NaN       NaN       NaN   
           air_0845d8395f30c6bb         NaN       NaN       NaN       NaN   
           air_084d98859256acf0         NaN       NaN       NaN       NaN   
           air_0867f7bebad6a649         NaN       NaN       NaN       NaN   
           air_08ba8cd01b3ba010    2.484907  2.484907  2.484907  2.484907   
           air_08cb3c4ee6cd6a22         NaN       NaN       NaN       NaN   
           air_08ef81d5b7a0d13f         NaN       NaN       NaN       NaN   
           air_08f994758a1e76d4         NaN       NaN       NaN       NaN   
...                                     ...       ...       ...       ...   
2016-01-03 air_f6b2489ccf873c3b         NaN       NaN       NaN       NaN   
           air_f6bfd27e2e174d16         NaN       NaN       NaN       NaN   
           air_f6cdaf7b7fdc6d78         NaN       NaN       NaN       NaN   
           air_f8233ad00755c35c         NaN       NaN       NaN       NaN   
           air_f85e21e543cf44f2         NaN       NaN       NaN       NaN   
           air_f88898cd09f40496         NaN       NaN       NaN       NaN   
           air_f911308e19d64236         NaN       NaN       NaN       NaN   
           air_f9168b23fdfc1e52         NaN       NaN       NaN       NaN   
           air_f927b2da69a82341         NaN       NaN       NaN       NaN   
           air_f957c6d6467d4d90         NaN       NaN       NaN       NaN   
           air_f96765e800907c77         NaN       NaN       NaN       NaN   
           air_fa12b40b02fecfd8         NaN       NaN       NaN       NaN   
           air_fa4ffc9057812fa2         NaN       NaN       NaN       NaN   
           air_fab092c35776a9b1    2.740319  2.740319  2.740319  2.740319   
           air_fb44f566d4f64a4e         NaN       NaN       NaN       NaN   
           air_fbadf737162a5ce3         NaN       NaN       NaN       NaN   
           air_fc477473134e9ae5         NaN       NaN       NaN       NaN   
           air_fcd4492c83f1c6b9         NaN       NaN       NaN       NaN   
           air_fcfbdcf7b1f82

In [114]:
daily_visits.loc[['air_04341b588bde96cd'],datetime(2016,1,1):datetime(2016,1,3)].apply(lambda x: np.nanmean(x),axis=1)

air_store_id
air_04341b588bde96cd    2.397895
dtype: float64